In [2]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [ ]:
#LIST_PATH
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

In [37]:
dataset[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/CT CT TAP 2.0 I30f 3',
 'CT',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/PT TEP TAP Corrig',
 'PT',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 '2017014',
 'cr_014_jul 25_2014.csv']

In [ ]:
#BUILD NIFTI FROM DATASET 1 2 et 3

json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_4/dataset_4.json'
dataset_4 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_4.append(info)

print(len(dataset_4))

#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', '78ca2911-9e9f-4f78-b80a-848024b95f92') 

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2', 'dataset', dataset)

In [40]:
#remove remove_bi_file

paths = get_series_path('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM')


In [43]:
from library_dicom.dicom_processor.tools.folders import *
for path in paths : 
    remove_bi_file(path)



In [39]:
#On met les erreurs de coté
#generer mip en meme temps 

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'



false_mask_dataset_1 = []
error_dataset_1 = []
path_mip = []

results_false_mask = []

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))
        subliste = []

        results = []

        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory+'/'+serie[-2], serie[]))
                #print("dicom effacé")

            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
                #print('dicom effacé')
            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)


0
MASK CORRECT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/PT p8273s0_FUE_ctac_resamp.img 3DRAMLA', 'PT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/CT', 'CT', '1.2.840.113704.1.111.920.1223892920.6', '2017031', 'cr_031_oct 13_2008.csv']
1
MASK CORRECT
EXPORT NIFTI PT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/CT CT TAP 3.0 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/PT PET TAP AC HD', 'PT', '1.2.840.113711.41813.3.11624.395627943.26.2116281012.16890', '2

In [45]:
print(len(error_dataset_1))
print(len(false_mask_dataset_1))

10
13


In [46]:
#SAVE THE SERIE WITH FALSE MASK 
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask', false_mask_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask_results', results_false_mask)

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'error', error_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP', 'path_mip', path_mip)

In [47]:
#create pdf 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [3]:
#REGENERATE MASK 
#READ CSV 


csv_pet0_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/GAINED_PET0_NIFTI.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [4]:
liste = []
for row in csv_data : 
    subliste = []
    subliste.append(row[0]) #patient id
    subliste.append(row[1]) #study uid
    subliste.append(row[-1]) #mask 

    liste.append(subliste)

In [4]:
len(liste)

573

In [5]:
#lire dataset et recuperer les csv 
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_1/dataset_1.json'
dataset_1 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_1.append(info)

print(len(dataset_1))


json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_2/dataset_2.json'
dataset_2 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_2.append(info)

print(len(dataset_2))

json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_3/dataset_3.json'
dataset_3 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_3.append(info)

print(len(dataset_3))

json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/unconstant_spacing/unconstant_spacing.json'
dataset_4 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_4.append(info)

print(len(dataset_4))

140
210
179
50


In [6]:
len(dataset_1)+len(dataset_2)+len(dataset_3)+len(dataset_4)

579

In [7]:
#ajouter csv pour unconstant_spacing/dataset_4

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/CSV41/PET0'
liste_csv = os.listdir(csv_directory)

In [8]:
dataset_4[0]

['/media/salim/DD 2To/GAINED_Validated_DICOMS/11011101031009/1.2.840.113619.2.55.3.698894.110.1378271612.544/1.2.840.113619.2.131.698894.1378366825.915943/1.2.840.113619.2.131.698894.1378366825.915943',
 'PT',
 '/media/salim/DD 2To/GAINED_Validated_DICOMS/11011101031009/1.2.840.113619.2.55.3.698894.110.1378271612.544/1.2.840.113619.2.55.3.698894.110.1378271612.548.3/1.2.840.113619.2.55.3.698894.110.1378271612.548.3',
 'CT',
 'review445.csv',
 '11011101031009']

In [9]:
for data in dataset_4 : 
    patient_id = data[5]
    for csv_file in liste_csv : 
        if patient_id in csv_file : 
            data.append(csv_file)


In [10]:
dataset = dataset_1 + dataset_2 + dataset_3 + dataset_4

print(len(dataset))

579


In [11]:
for mask in liste : 
    #print(liste.index(mask))
    for serie in dataset :
        #print(dataset.index(serie))
        if mask[1] in serie[0] : #if study id dans la serie => bonne série
            #print(serie)
            mask.append(serie[6]) #on rajoute csv apres

In [12]:
print(len(liste))

573


In [14]:
liste[1]

['11011101021002',
 '1.2.250.1.59.470.940100027.1.20130128113103.7763.17688',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011101021002/pet0/1.2.250.1.59.470.940100027.1.20130128113103.7763.17688_nifti_mask.nii',
 'patient_11011101021002_PET_0_review_18_41.csv']

In [13]:
csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/CSV41/PET0'
for mask in liste : 
    print(mask[0])
    pixel_spacing, origin, direction, size = read_mask_nifti(mask[2])
    mask_objet = MaskBuilder(os.path.join(csv_directory, mask[3]), (size[0], size[1], size[2]))
    mask_4D = mask_objet.mask_array
    print(mask_4D.shape)

    build_nifti_mask(mask_4D, mask[2], pixel_spacing, origin, direction)



11011101021001
(144, 144, 234, 9)
11011101021002
(144, 144, 255, 49)
11011101021003
(144, 144, 255, 12)
11011101021004
(144, 144, 234, 3)
11011101021005
(144, 144, 276, 5)
11011101021006
(144, 144, 234, 2)
11011101021007
(144, 144, 255, 3)
11011101021008
(144, 144, 276, 2)
11011101021009
(144, 144, 255, 4)
11011101021010
(144, 144, 234, 4)
11011101021011
(144, 144, 255, 2)
11011101021012
(144, 144, 275, 14)
11011101021013
(144, 144, 234, 1)
11011101021014
(144, 144, 276, 15)
11011101031001
(200, 200, 481, 9)
11011101031002
(144, 144, 255, 18)
11011101031003
(256, 256, 515, 2)
11011101031004
(144, 144, 276, 58)
11011101031005
(200, 200, 246, 70)
11011101031006
(144, 144, 254, 13)
11011101031007
(144, 144, 234, 7)
11011101031008
(144, 144, 276, 11)
11011101031009
(128, 128, 313, 3)
11011101031011
(200, 200, 543, 24)
11011101031012
(144, 144, 234, 11)
11011101031013
(256, 256, 263, 62)
11011101031015
(256, 256, 299, 47)
11011101031016
(200, 200, 915, 18)
11011101051001
(168, 168, 390, 4)


In [15]:
liste[12]

['11011101021013',
 '1.2.250.1.59.470.940100027.1.20141009123449.929.131461',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011101021013/pet0/1.2.250.1.59.470.940100027.1.20141009123449.929.131461_nifti_mask.nii',
 'patient_11011101021013_PET_0_review_92_41.csv']

In [17]:
direction = read_mask_nifti(liste[12][2])[2]
direction

(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)

In [18]:
if len(direction) != 9 : 
    print('mask 4D')

In [40]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/false_mask.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

13


In [42]:
serie = dataset[9]
print(serie)

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/CT CT TAP 55 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/PT PET TAP AC HD', 'PT', '2.16.840.1.113669.632.20.310000.20001712333', '2017097', 'cr_097_dec 22_2009.csv', 'cr_097_dec 7_2011.csv']


In [44]:
serie_pt_objet = SeriesPT(serie[2])
nifti_array = serie_pt_objet.get_numpy_array()
study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

size_matrix = serie_pt_objet.get_size_matrix()
mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)



In [45]:
mask_objet.ecart_suv_max(nifti_array)

[1, 'POLYGON', 0.7599999999999998]

In [46]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/false_mask.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

14


In [47]:
#problème de bords pas pris en compte => mip ok donc on genere nifti
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print("dicom effacé")

            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print('dicom effacé')
            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)

0
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
5
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
10
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
11
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
12
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
13
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK


In [ ]:
#GENERER MIP DES MASK FAUX
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_MIP/DATASET_1'

path_mip = []
for serie in dataset_coronal : 
    index = dataset_coronal.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        flip_mask = mask_objet.flip_z(mask_4D)
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage

        flip_mask = mask_objet.flip_z(mask_4D)
        
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


In [ ]:
#lire json false_mask 

In [ ]:
#GENERER MIP MASK CORONAL 

In [6]:
#ALGO MIP FLIP 

mip_directory = '/media/salim/DD 2To/AHL2011_MIP_FLIP'

path_mip = []
for serie in dataset_coronal : 
    index = dataset_coronal.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        flip_mask = mask_objet.flip_z(mask_4D)
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage

        flip_mask = mask_objet.flip_z(mask_4D)
        
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)

0
(144, 144, 234, 21)
(144, 144, 234, 21)
taille mask :  (144, 144, 234, 21)
1
(256, 256, 299, 11)
(256, 256, 299, 11)
taille mask :  (256, 256, 299, 11)
2
(144, 144, 255, 4)
(144, 144, 255, 4)
taille mask :  (144, 144, 255, 4)
3
(256, 256, 335, 25)
(256, 256, 335, 25)
taille mask :  (256, 256, 335, 25)
4
(256, 256, 515, 12)
(256, 256, 515, 12)
taille mask :  (256, 256, 515, 12)
5
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
6
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
7
(144, 144, 255, 15)
(144, 144, 255, 15)
taille mask :  (144, 144, 255, 15)
8
(144, 144, 234, 11)
(144, 144, 234, 11)
taille mask :  (144, 144, 234, 11)
9
(168, 168, 405, 32)
(168, 168, 405, 32)
taille mask :  (168, 168, 405, 32)
10
(144, 144, 255, 5)
(144, 144, 255, 5)
taille mask :  (144, 144, 255, 5)


In [3]:
#POUR PET 2 ET PET 4 (SANS MASK)

#RELOAD JSON FROM PARSED DICOM IPYNB


json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4/GAINED_list_path_CT_PET4.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

534


In [4]:
dataset[500]

['/media/salim/DD 2To/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578',
 'CT',
 '/media/salim/DD 2To/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338',
 'PT',
 '1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006',
 '51011101881013']

In [5]:
#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', 'Elements') 

In [6]:
dataset[500]

['/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578/1.3.12.2.1107.5.1.4.1007.30000014021712362196800004578',
 'CT',
 '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881013/1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338/1.3.12.2.1107.5.1.4.11087.30000014021708361704600036338',
 'PT',
 '1.3.12.2.1107.5.1.4.1007.30000014021712095628100000006',
 '51011101881013']

In [ ]:
#CHECKER LES UNCONSTANT SPACING
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [7]:
#GENERATION NIFTI CT PET DES PET 2/4

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'

error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")



        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

15600017415/1.3.12.2.1107.5.1.4.11087.30000014091608515915600017415', 'PT', '1.3.12.2.1107.5.1.4.11087.30000014091608334298400000029', '51011101881020']
[Errno 2] No such file or directory: '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881020/1.3.12.2.1107.5.1.4.11087.30000014091608334298400000029/1.3.12.2.1107.5.1.4.11087.30000014091608515915600017415/1.3.12.2.1107.5.1.4.11087.30000014091608515915600017415'
508
['/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881021/1.2.250.1.199.10.90001.90158.20141020082205/1.3.12.2.1107.5.1.4.1007.30000014102008020323400000172/1.3.12.2.1107.5.1.4.1007.30000014102008020323400000172', 'CT', '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881021/1.2.250.1.199.10.90001.90158.20141020082205/1.3.12.2.1107.5.1.4.1007.30000014102008022110900000839/1.3.12.2.1107.5.1.4.1007.30000014102008022110900000839', 'PT', '1.2.250.1.199.10.90001.90158.20141020082205', '51011101881021']
[Errno 2] No such file or directory: '/m

unconstant_spacing = []
serie_error = []
for serie in error : 
    print(error.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            print("PET")
            nifti_array = serie_pt_objet.get_numpy_array()
            print(serie_pt_objet.get_z_spacing())
            print(serie_pt_objet.calculate_z_spacing(round_ = True))
            print("CT")
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            print(serie_ct_objet.get_z_spacing())
            print(serie_ct_objet.calculate_z_spacing(round_ = True))
                

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            print("PET")
            nifti_array = serie_pt_objet.get_numpy_array()
            print(serie_pt_objet.get_z_spacing())
            print(serie_pt_objet.calculate_z_spacing(round_ = True))
            print("CT")
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            print(serie_ct_objet.get_z_spacing())
            print(serie_ct_objet.calculate_z_spacing(round_ = True))
    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)
